# Bayesian Statistics

While preparing codes, I have utilized following source: https://app.datacamp.com/learn/courses/bayesian-data-analysis-in-python

In [ ]:
#pip install pymc3

In [ ]:
import pymc3 as pm

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
iris = sns.load_dataset('iris')

In [ ]:
pd.options.display.max_rows = 50

The Iris flower data set is a multivariate data set introduced by the British statistician and biologist Ronald Fisher in his 1936 paper The use of multiple measurements in taxonomic problems. It is sometimes called Anderson's Iris data set because Edgar Anderson collected the data to quantify the morphologic variation of Iris flowers of three related species. The data set consists of 50 samples from each of three species of Iris (Iris Setosa, Iris virginica, and Iris versicolor). Four features were measured from each sample: the length and the width of the sepals and petals, in centimeters.

In [ ]:
iris

In [ ]:
formula = "sepal_length ~ sepal_width"

with pm.Model() as model_1:
    
    pm.GLM.from_formula(formula, data=iris)
    trace_1 = pm.sample(draws=1000, tune=500, chains=12)

In [ ]:
pm.traceplot(trace_1 ,figsize=(10,10))

In [ ]:
pm.forestplot(trace_1, figsize=(10,10))

For other arguments of pmyc3 plots, please visit: https://pymc3-testing.readthedocs.io/en/rtd-docs/api/plots.html

In [ ]:
pm.summary(trace_1)

#### For the intercept (as an example):
- **mean**: The mean (average) value of the intercept is 6.522.
- **sd (Standard Deviation)**: The standard deviation of the intercept is 0.491, indicating the spread of the intercept's values.
- **hdi_3%**: The lower bound of the Highest Density Interval (HDI) for the intercept is 5.626. This represents the lower end of the interval within which 94% of the values lie.
- **hdi_97%**: The upper bound of the HDI for the intercept is 7.466. This represents the upper end of the interval within which 94% of the values lie.
- **mcse_mean (Monte Carlo Standard Error for mean)**: The MCSE for the mean of the intercept is 0.008, indicating the uncertainty in the estimate of the mean.
- **mcse_sd (Monte Carlo Standard Error for standard deviation)**: The MCSE for the standard deviation of the intercept is 0.005.
- **ess_bulk (Effective Sample Size for bulk)**: The ESS for the bulk of the distribution for the intercept is 4053.0, indicating the number of effective samples used to calculate the bulk of the distribution.
- **ess_tail (Effective Sample Size for tail)**: The ESS for the tail of the distribution for the intercept is 4153.0.
- **r_hat**: The potential scale reduction factor on split chains (r_hat) for the intercept is 1.0, indicating convergence of the chains (values close to 1.0 indicate good mixing and convergence).


**Bulk**: This usually refers to the central mass or the main body of the distribution. Effective Sample Size (ESS) for the bulk provides a measure of how many independent and identically distributed samples would be equivalent to the correlated samples in the central part of the posterior distribution. It is a way to quantify the amount of "information" that the sample contains about the central tendency of the parameter's distribution.

**Tail**: This refers to the outer sections of the distribution, away from the median, often associated with the extremes or the tails of the distribution. The ESS for the tail provides a measure of the number of independent samples equivalent to the correlated samples in the tails of the posterior distribution. It is important for understanding the uncertainty in the extremes of the distribution, which can be critical for evaluating risks or rare events.

In Bayesian analysis, having a high ESS for both the bulk and the tail of the distribution is important because it indicates that the Markov Chain Monte Carlo (MCMC) simulation has effectively explored the parameter space, providing a reliable approximation of the posterior distribution.

In [ ]:
formula = "sepal_length ~ petal_length + petal_width"

with pm.Model() as model_2:
    
    pm.GLM.from_formula(formula, data=iris)
    trace_2 = pm.sample(draws=1000, tune=500, chains=12)

In [ ]:
pm.traceplot(trace_2 ,figsize=(12,12))

In [ ]:
pm.forestplot(trace_2, figsize=(10,10))

In [ ]:
pm.summary(trace_2)

In [ ]:
#model1_loo = az.loo(trace_1, model_1)
#model2_loo = az.loo(trace_2, model_2)
#df_comp_loo = az.compare({"model_1": trace_1, "model_2": trace_2})
#az.plot_compare(df_comp_loo, insample_dev=False);

In [ ]:
formula = "sepal_length ~ sepal_width"

with pm.Model() as model_3:
    
    pm.GLM.from_formula(formula, data=iris)
    trace_3 = pm.sample(draws=1000, tune=500, chains=12)
    posterior_predictive = pm.fast_sample_posterior_predictive(trace_3)

In [ ]:
print(posterior_predictive['y'])

#### Now we have a distribution for each observation

In [ ]:
pm.plot_posterior(posterior_predictive['y'][:,0])

In [ ]:
pm.plot_posterior(posterior_predictive['y'][:,119])

In [ ]:
len(posterior_predictive['y'][:,119]) #12000 = 1000 draws * 12 chains

In [ ]:
errors=[]
for index, observation in iris.iterrows():
    error=posterior_predictive['y'][:,index] - observation['sepal_length']
    errors.append(error)
    
error_distribution=np.array(errors).reshape(-1)
error_distribution.shape

In [ ]:
# Plot the error distribution
pm.plot_posterior(error_distribution)
plt.show()

### Assign prior distributions

https://docs.pymc.io/en/latest/api/distributions/generated/pymc.Binomial.html

In [ ]:
Y = iris['sepal_length']
X1 = iris['sepal_width']
X2 = iris['petal_length']
X3 = iris['petal_width']

with pm.Model() as model_4:
    beta0 = pm.Normal("beta0", 0.0, 1.0) #Change to uniform distribution to see what happens (gamma, halfnormal, uniform, etc.)
    beta1 = pm.Normal("beta1", 0.0, 1.0)
    beta2 = pm.Normal("beta2", 0.0, 1.0)
    beta3 = pm.Normal("beta3", 0.0, 1.0)

    mean_y = beta0 + beta1 * X1 + beta2 * X2 + beta3 * X3
 
    pm.Normal("Y_obs", mu=mean_y, sigma=1, observed=Y)
    trace_4 = pm.sample(draws=100, tune=50, chains=2)
    posterior_predictive = pm.fast_sample_posterior_predictive(trace_4)

In [ ]:
pm.traceplot(trace_4 ,figsize=(15,15))

The prior belief changes the outcome. That's how, we think of Bayes as a degree of belief

#### Updating priors


https://docs.pymc.io/en/v3/pymc-examples/examples/pymc3_howto/updating_priors.html

We want to keep probability density of posterior draws, following function does so, by using kernel density estimation. Function takes samples, and convert them into distributions.

In [ ]:
def from_posterior(param, samples):
    smin, smax = np.min(samples), np.max(samples)
    width = smax - smin
    x = np.linspace(smin, smax, 100)
    y = stats.gaussian_kde(samples)(x)

    # what was never sampled should have a small probability but not 0,
    # so we'll extend the domain and use linear approximation of density on it
    
    x = np.concatenate([[x[0] - 3 * width], x, [x[-1] + 3 * width]])
    y = np.concatenate([[0], y, [0]])
    return Interpolated(param, x, y)

Store means of coefficients, from the posterior draws of the model above.

In [ ]:
beta0_true = np.mean(trace_4.get_values('beta0'))
beta1_true = np.mean(trace_4.get_values('beta1'))
beta2_true = np.mean(trace_4.get_values('beta2'))
beta3_true = np.mean(trace_4.get_values('beta3'))

print(beta0_true,beta1_true,beta2_true,beta3_true)

In [ ]:
len(trace_4.get_values('beta0'))

Now we will use posterior draws of previous model as priors of a new model.

In [ ]:
from pymc3 import Model, Normal, Slice, sample
from scipy import stats
from pymc3.distributions import Interpolated

traces =  [trace_4]

model = Model()
with model:
    # Priors are posteriors from previous iteration
    beta0 = from_posterior("beta0", trace_4["beta0"])
    beta1 = from_posterior("beta1", trace_4["beta1"])
    beta2 = from_posterior("beta2", trace_4["beta2"])
    beta3 = from_posterior("beta3", trace_4["beta3"])

    # Expected value of outcome
    mean_y = beta0 + beta1 * X1 + beta2 * X2 + beta3 * X3

    # Likelihood (sampling distribution) of observations
    Y_obs = Normal("Y_obs", mu=mean_y, sigma=1, observed=Y)

    # draw 100 posterior samples
    trace = pm.sample(draws=100, tune=50, chains=2)

    traces.append(trace)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

print("Posterior distributions after " + str(len(traces)) + " iterations.")
cmap = mpl.cm.autumn
for param in ["beta0", "beta1", "beta2", "beta3"]:
    plt.figure(figsize=(8, 2))
    for update_i, trace in enumerate(traces):
        samples = trace[param]
        smin, smax = np.min(samples), np.max(samples)
        x = np.linspace(smin, smax, 100)
        y = stats.gaussian_kde(samples)(x)
        plt.plot(x, y, color=cmap(1 - update_i / len(traces))) #in each step color gets from yellow to red
    plt.axvline({"beta0": beta0_true, "beta1": beta1_true, "beta2": beta2_true, "beta3": beta3_true}[param], c="k")
    plt.ylabel("Frequency")
    plt.title(param)

plt.tight_layout()

In [ ]:
for i in range(1,10):

    model = Model()
    with model:
        # Priors are posteriors from previous iteration
        beta0 = from_posterior("beta0", traces[i]["beta0"])
        beta1 = from_posterior("beta1", traces[i]["beta1"])
        beta2 = from_posterior("beta2", traces[i]["beta2"])
        beta3 = from_posterior("beta3", traces[i]["beta3"])

        # Expected value of outcome
        mean_y = beta0 + beta1 * X1 + beta2 * X2 + beta3 * X3

        # Likelihood (sampling distribution) of observations
        Y_obs = Normal("Y_obs", mu=mean_y, sigma=1, observed=Y)

        # draw 10000 posterior samples
        trace = pm.sample(draws=100, tune=50, chains=2)
        traces.append(trace)

In [ ]:
beta0_true = np.mean(traces[1].get_values('beta0'))
beta1_true = np.mean(traces[1].get_values('beta1'))
beta2_true = np.mean(traces[1].get_values('beta2'))
beta3_true = np.mean(traces[1].get_values('beta3'))


print("Posterior distributions after " + str(len(traces)) + " iterations.")
cmap = mpl.cm.autumn
for param in ["beta0", "beta1", "beta2", "beta3"]:
    plt.figure(figsize=(8, 2))
    for update_i, trace in enumerate(traces):
        samples = trace[param]
        smin, smax = np.min(samples), np.max(samples)
        x = np.linspace(smin, smax, 100)
        y = stats.gaussian_kde(samples)(x)
        plt.plot(x, y, color=cmap(1 - update_i / len(traces))) #in each step color gets from yellow to red
    plt.axvline({"beta0": beta0_true, "beta1": beta1_true, "beta2": beta2_true, "beta3": beta3_true}[param], c="k")
    plt.ylabel("Frequency")
    plt.title(param)

plt.tight_layout()

In [ ]:
traces

In [ ]:
traces[1].varnames


In [ ]:
final_trace = traces[-1]


In [ ]:
pm.summary(final_trace)


#### Prediction

In [ ]:
intercept_mean = np.mean(final_trace.get_values('beta0')) 
beta_1_mean = np.mean(final_trace.get_values('beta1')) 
beta_2_mean = np.mean(final_trace.get_values('beta2')) 
beta_3_mean = np.mean(final_trace.get_values('beta3')) 

In [ ]:
predicted_Y = {}
for X1_dynamic in range(0,5):
    pred_mean = intercept_mean + beta_1_mean * X1_dynamic + beta_2_mean * X2.mean() + beta_3_mean * X3.mean()
    Y_pred = np.random.normal(pred_mean, sd_mean, size=100)
    predicted_Y.update({X1_dynamic: Y_pred})

In [ ]:
# Draw a forest plot of predicted Y values
pm.forestplot(predicted_Y)
plt.show()

In [ ]:
print(pm.hdi(predicted_Y[4], hdi_prob=0.94)) #Change the hdi limitsY